In [1]:
from pyspark import SparkContext
from geopyspark.geopycontext import GeoPyContext
from geopyspark.geotrellis.catalog import read, read_value, query, write
from geopyspark.geotrellis.constants import SPATIAL, ZOOM, TILE
from geopyspark.geotrellis.geotiff_rdd import get
from geopyspark.geotrellis.rdd import RasterRDD, TiledRasterRDD
from geonotebook.vis.geotrellis.render_methods import single_band_render_from_color_map
from geonotebook.wrappers import GeoTrellisCatalogLayerData, RddRasterData
import numpy as np

In [2]:
catalog_uri = "file://home/hadoop/notebooks/catalog"

In [ ]:
# Gather GeoTiffs

!curl -o /tmp/DevelopedLand.tiff https://s3.amazonaws.com/chattademo/DevelopedLand.tiff
!curl -o /tmp/FarmlandOrForestedLandsWithPrimeAgriculturalSoils.tiff https://s3.amazonaws.com/chattademo/FarmlandOrForestedLandsWithPrimeAgriculturalSoils.tiff
!curl -o /tmp/FarmlandWithoutPrimeAgriculturalSoils.tiff.tiff https://s3.amazonaws.com/chattademo/FarmlandWithoutPrimeAgriculturalSoils.tiff.tiff
!curl -o /tmp/ForestedLand.tiff https://s3.amazonaws.com/chattademo/ForestedLands.tiff
!curl -o /tmp/ImperviousSurfacesBarrenLandsOpenWater.tiff https://s3.amazonaws.com/chattademo/ImperviousSurfacesBarrenLandsOpenWater.tiff
!curl -o /tmp/NonWorkingProtectedOrPublicLands.tiff https://s3.amazonaws.com/chattademo/NonWorkingProtectedOrPublicLands.tiff
!curl -o /tmp/PrimeAgriculturalSoilsNotForestedOrFarmland.tiff https://s3.amazonaws.com/chattademo/PrimeAgriculturalSoilsNotForestedOrFarmland.tiff
!curl -o /tmp/PrivatelyOwnedWorkingLandsWithEasements.tiff https://s3.amazonaws.com/chattademo/PrivatelyOwnedWorkingLandsWithEasements.tiff
!curl -o /tmp/PublicallyOwnedWorkingLands.tiff https://s3.amazonaws.com/chattademo/PublicallyOwnedWorkingLands.tiff
!curl -o /tmp/Wetlands.tiff https://s3.amazonaws.com/chattademo/Wetlands.tiff


In [3]:
layers = { "developed-land" : "file:///tmp/DevelopedLand.tiff",
           "prime-ag-soil" : "file:///tmp/FarmlandOrForestedLandWithPrimeAgriculturalSoils.tiff",
           "not-prime-ag-soil" : "file:///tmp/FarmlandOrForestedLandsWithPrimeAgriculturalSoils.tiff",
           "forested-lands" : "file:///tmp/ForestedLands.tiff",
           "impervious" : "file:///tmp/ImperviousSurfacesBarrenLandsOpenWater.tiff",
           "protected" : "file:///tmp/NonWorkingProtectedOrPublicLands.tiff",
           "prime-ag-soil-notfarm" : "file:///tmp/PrimeAgriculturalSoilsNotForestedOrFarmland.tiff",
           "private-with-easements" : "file:///tmp/PrivatelyOwnedWorkingLandsWithEasements.tiff",
           "public-working-lands" : "file:///tmp/PublicallyOwnedWorkingLands.tiff",
           "wetlands" : "file:///tmp/Wetlands.tiff" }

In [4]:
sc = SparkContext(appName="Chatta Ingest")
geopysc = GeoPyContext(sc)

In [5]:
def ingest_layer(layer_name, path):
    rdd = get(geopysc, SPATIAL, path, maxTileSize=512, numPartitions=20)
    metadata = rdd.collect_metadata(tile_size=512)
    metadata['cellType'] = "uint8"
    laid_out = rdd.tile_to_layout(metadata)
    reprojected = laid_out.reproject("EPSG:3857", scheme=ZOOM)
    for layer_rdd in reprojected.pyramid(reprojected.zoom_level, 0):
        write(catalog_uri, layer_name, layer_rdd)    

In [6]:
for layer in layers:
    print("Ingesting %s at %s" % (layer, layers[layer]))
    ingest_layer(layer, layers[layer])


Ingesting not-prime-ag-soil at file:///tmp/FarmlandOrForestedLandsWithPrimeAgriculturalSoils.tiff


In [ ]:
ingest_layer("developed-land", layers['developed-land'])

In [7]:
M.set_center(-85.2935099666654, 35.20529964318362, 9)

---MESSAGE---
<class 'dict'>
{"params": [{"required": true, "value": -85.2935099666654, "key": "x"}, {"required": true, "value": 35.20529964318362, "key": "y"}, {"required": true, "value": 9, "key": "z"}], "id": "31b94a51-936d-418e-a597-dc7b660839ce", "method": "set_center", "jsonrpc": "2.0"}
---END MESSAGE---


In [8]:
cmap = { 0 : "#00000000", 1: "#FF9D57FF" }
cmap_render = single_band_render_from_color_map(cmap)
def render_tile(tile):
    arr = tile[0]
    return cmap_render(arr)
data = GeoTrellisCatalogLayerData(geopysc, 
                                  catalog_uri, 
                                  "not-prime-ag-soil",
                                  SPATIAL)
M.add_layer(data, render_tile=render_tile)

---MESSAGE---
<class 'dict'>
{"params": [{"required": true, "value": "-1759210083788317921__not-prime-ag-soil", "key": "layer_name"}, {"required": true, "value": "http://localhost:8000/user/hadoop/geotrellis/53934", "key": "vis_url"}, {"required": true, "value": {"opacity": 1.0, "layer_type": null, "interval": null, "zIndex": 0, "attribution": null, "colormap": [], "projection": "EPSG:3857", "kernel_id": "4fa43155-de2e-4cd4-9103-a1556158eb88", "gamma": 1.0}, "key": "vis_params"}, {"required": true, "value": {}, "key": "query_params"}], "id": "b7aa6ed5-f09b-43f9-8a92-8b0b536f651e", "method": "add_layer", "jsonrpc": "2.0"}
---END MESSAGE---


::1 - - [2017-05-14 00:20:13] "GET /tile/9/134/202.png HTTP/1.1" 200 8522 0.389050
::1 - - [2017-05-14 00:20:13] "GET /tile/9/135/202.png HTTP/1.1" 200 9537 0.176852
::1 - - [2017-05-14 00:20:14] "GET /tile/9/134/201.png HTTP/1.1" 200 3527 0.116992
::1 - - [2017-05-14 00:20:14] "GET /tile/9/134/203.png HTTP/1.1" 200 7460 0.127817
::1 - - [2017-05-14 00:20:14] "GET /tile/9/135/201.png HTTP/1.1" 200 4514 0.106432
::1 - - [2017-05-14 00:20:14] "GET /tile/9/135/203.png HTTP/1.1" 200 3533 0.109641
::1 - - [2017-05-14 00:20:14] "GET /tile/9/136/202.png HTTP/1.1" 200 583 0.160028
::1 - - [2017-05-14 00:20:14] "GET /tile/9/133/202.png HTTP/1.1" 200 1823 0.113096
::1 - - [2017-05-14 00:20:14] "GET /tile/9/133/201.png HTTP/1.1" 200 436 0.150071
::1 - - [2017-05-14 00:20:15] "GET /tile/9/133/203.png HTTP/1.1" 200 10921 0.109407
::1 - - [2017-05-14 00:20:15] "GET /tile/9/136/201.png HTTP/1.1" 200 436 0.103508
::1 - - [2017-05-14 00:20:15] "GET /tile/9/136/203.png HTTP/1.1" 200 436 0.114002
::1 - -

::1 - - [2017-05-14 00:20:30] "GET /tile/13/2141/3251.png HTTP/1.1" 200 4786 0.093942
::1 - - [2017-05-14 00:20:30] "GET /tile/13/2145/3256.png HTTP/1.1" 200 6340 0.092449
::1 - - [2017-05-14 00:20:32] "GET /tile/16/17156/26021.png HTTP/1.1" 500 4136 0.011108
::1 - - [2017-05-14 00:20:32] "GET /tile/16/17156/26022.png HTTP/1.1" 500 4136 0.005935
::1 - - [2017-05-14 00:20:32] "GET /tile/16/17157/26021.png HTTP/1.1" 500 4136 0.003362
::1 - - [2017-05-14 00:20:32] "GET /tile/16/17157/26022.png HTTP/1.1" 500 4136 0.006135
::1 - - [2017-05-14 00:20:32] "GET /tile/16/17155/26021.png HTTP/1.1" 500 4136 0.006384
::1 - - [2017-05-14 00:20:32] "GET /tile/16/17156/26023.png HTTP/1.1" 500 4136 0.005192
::1 - - [2017-05-14 00:20:32] "GET /tile/16/17155/26011.png HTTP/1.1" 500 4136 0.006772
::1 - - [2017-05-14 00:20:32] "GET /tile/16/17155/26012.png HTTP/1.1" 500 4136 0.005203
::1 - - [2017-05-14 00:20:32] "GET /tile/16/17156/26011.png HTTP/1.1" 500 4136 0.004818
::1 - - [2017-05-14 00:20:32] "GET /

::1 - - [2017-05-14 00:20:39] "GET /tile/13/2145/3249.png HTTP/1.1" 200 5094 0.149409
::1 - - [2017-05-14 00:20:39] "GET /tile/13/2143/3249.png HTTP/1.1" 200 5104 0.099926
::1 - - [2017-05-14 00:20:39] "GET /tile/14/4291/6504.png HTTP/1.1" 500 4136 0.004362
::1 - - [2017-05-14 00:20:39] "GET /tile/14/4287/6505.png HTTP/1.1" 500 4136 0.004162
::1 - - [2017-05-14 00:20:39] "GET /tile/14/4290/6505.png HTTP/1.1" 500 4136 0.005837
::1 - - [2017-05-14 00:20:39] "GET /tile/14/4291/6501.png HTTP/1.1" 500 4136 0.003030
::1 - - [2017-05-14 00:20:39] "GET /tile/14/4290/6500.png HTTP/1.1" 500 4136 0.005362
::1 - - [2017-05-14 00:20:39] "GET /tile/14/4286/6504.png HTTP/1.1" 500 4136 0.003485
::1 - - [2017-05-14 00:20:39] "GET /tile/14/4286/6501.png HTTP/1.1" 500 4136 0.005452
::1 - - [2017-05-14 00:20:39] "GET /tile/14/4286/6505.png HTTP/1.1" 500 4136 0.003159
::1 - - [2017-05-14 00:20:39] "GET /tile/14/4291/6500.png HTTP/1.1" 500 4136 0.005346
::1 - - [2017-05-14 00:20:39] "GET /tile/14/4286/6500.

In [ ]:
M.remove_layer(M.layers[0])